In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
df=pd.read_csv('train.csv')
df.head()

FileNotFoundError: [Errno 2] File train.csv does not exist: 'train.csv'

In [ ]:
df.dtypes

In [ ]:
#df_corr=abs(df.corr()) < 0.25
df.corr()

In [ ]:
list(df.columns.values)

In [ ]:

X = df.drop(['Survived'], axis=1)

y = df['Survived']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
m = LogisticRegression()

### Feature Engineering

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
MEDIAN_AGE_FROM_TRAINING = X_train['Age'].median()
MODE_EMBARKED_FROM_TRAINING = X_train['Embarked'].mode().values[0]
MEDIAN_FARE_FROM_TRAINING = X_train['Fare'].median()

def feature_engineer(df):
    
    # Fill in missing values for Age
    df = df.fillna({'Age': MEDIAN_AGE_FROM_TRAINING})    
# Age groups 0-20(under_20) / 20-40(20_to_39) / 40-60(40_to_59) / 60+(60_and_more) 
    df['Age'] = ['under_20' if x<20.0 else '20_to_39' if 20.0<=x<40.0 else '40_to_59' if 40.0<=x<60.0 else '60_and_more' for x in df['Age']] 

    # Fill in missing values for Embarked
    df = df.fillna({'Embarked': MODE_EMBARKED_FROM_TRAINING})

# Male/Female = 0/1
    df['Sex'].replace({'male' : 0, 'female' : 1}, inplace=True)

    # Fill in missing values for Fare
    df = df.fillna({'Fare': MEDIAN_FARE_FROM_TRAINING})
    
# Class groups '1st' / '2nd' / '3rd' 
    df['Pclass'].replace({1 : '1st', 2 : '2nd', 3:'3rd'}, inplace=True)
    
# New column 'SibSp_Parch' = 'SibSp' * 'Parch'
    df['SibSp_Parch']=df['SibSp'] * df['Parch']
    
    df_final = df[['Sex', 'Age', 'SibSp_Parch','Pclass']]
        
    # Label encoding / one-hot encoding, create binary columns out of original categories
    df_final = pd.get_dummies(df_final, drop_first=True)
        
    # Return result   
    return df_final

In [ ]:
X_train_fe = feature_engineer(X_train)

In [ ]:
X_test_fe = feature_engineer(X_test)

In [ ]:
m.fit(X_train_fe, y_train)

In [ ]:
m.score(X_train_fe, y_train)

In [ ]:
m.score(X_test_fe, y_test)

In [ ]:
# df[['Sex', 'Age', 'SibSp_Parch','Pclass']]
# Kaggle score: 0.76555

X = df[['Sex', 'Pclass', 'Parch','SibSp', 'Fare']]  
m.score(X_train_fe, y_train)  
0.8008982035928144  
m.score(X_test_fe, y_test)  
0.7892376681614349

X = df[['Age', 'Sex', 'Parch', 'SibSp','Parch','Embarked','Pclass']]  
m.score(X_train_fe, y_train)  
0.8023952095808383  
m.score(X_test_fe, y_test)  
0.8071748878923767

X = df[['Age', 'Sex', 'Pclass', 'SibSp','Parch','Fare']]  
m.score(X_train_fe, y_train)  
0.7994011976047904  
m.score(X_test_fe, y_test)  
0.7982062780269058

 = df[['Pclass', 'Parch','Sex']]  
m.score(X_train_fe, y_train)  
0.7949101796407185  
m.score(X_test_fe, y_test)  
0.7847533632286996X

X = df[['Pclass', 'Sex', 'Embarked', 'Age']]  
m.score(X_train_fe, y_train)  
0.7919161676646707  
m.score(X_test_fe, y_test)  
0.7892376681614349  

X = df[['Pclass', 'Sex']]  
m.score(X_train_fe, y_train)  
0.7874251497005988  
m.score(X_test_fe, y_test)  
0.7847533632286996

X = df[['Pclass', 'Sex', 'Age']]  
m.score(X_train_fe, y_train)  
0.7949101796407185  
m.score(X_test_fe, y_test)  
0.7982062780269058

X = df[['Age', 'Sex', 'Fare']]  
m.score(X_train_fe, y_train)  
0.7874251497005988  
m.score(X_test_fe, y_test)  
0.7713004484304933  

X = df[['Age', 'Sex', 'Parch']]  
m.score(X_train_fe, y_train)  
0.7889221556886228  
m.score(X_test_fe, y_test)  
0.7847533632286996

### Predict

In [ ]:
X_kaggle=pd.read_csv('test.csv')
X_kaggle.head()

In [ ]:
X_kaggle_fe = feature_engineer(X_kaggle)
X_kaggle_fe

In [ ]:
ypred = m.predict(X_kaggle_fe)
ypred.shape

In [ ]:
df_final=pd.DataFrame({'PassengerId':X_kaggle['PassengerId'], 'Survived':ypred})
df_final.set_index('PassengerId', inplace=True)
df_final

In [ ]:
#df_final.to_csv('df_final_andrey')

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_results = cross_val_score(m, X_train_fe, y_train, cv=5, scoring='accuracy')
cv_results

In [ ]:
cv_results.mean()
# we would expect that this roughly corresponds to the final testing score.
# if training score >> mean(validation scores), then you are overfitting!

In [ ]:
cv_results.std()
# To check your model's variance, compute the standard deviation of all the scores. 
# If this variance is HIGH, it means your model's performance varies a lot / depends a lot on the sampling.